# Generate ChordPro PDFs

- input/songs/sections/
    - Testlieder
- output/chordpro-pdfs/sections/
    - Testlieder

In [ ]:
# https://ipython.readthedocs.io/en/stable/config/options/terminal.html#configtrait-InteractiveShell.ast_node_interactivity
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # last_expr, all

In [ ]:
import os
import sys
import fnmatch
import subprocess
import pandas as pd
import re
import pprint

In [ ]:
try:
   import cPickle as pickle
except:
   import pickle

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
path_input_chordpro_sections = os.path.normpath("../data/input/songs/sections/")
path_output_chordpro_pdfs_sections = os.path.normpath("../data/output/chordpro-pdfs/sections/")
path_input_chordpro_sections
path_output_chordpro_pdfs_sections

In [ ]:
# https://stackoverflow.com/a/59938961 (always use scandir)
song_sections = [{"name": f.name, "path": f.path} for f in os.scandir(path_input_chordpro_sections) if f.is_dir()]
#song_sections_paths = [f.path for f in os.scandir(path_input_sections) if f.is_dir()]
song_sections

In [ ]:
for section in song_sections:
    
    # python list comprehension tips: multiple if statements, multiple lines syntax
    # https://stackoverflow.com/a/15248356
    # https://stackoverflow.com/a/12372259
    songs_of_section = [
        {
            "name": f.name,
            "path": f.path
        } 
        for f in os.scandir(section['path'])
            if f.is_file()
            if fnmatch.fnmatch(f,'*.cho')
    ]
    
    section["songs"] = songs_of_section

In [ ]:
# https://stackoverflow.com/questions/40972805/python-capture-contents-inside-curly-braces/40972959
# https://stackoverflow.com/questions/11310567/python-re-match-string-in-a-file/11310926

regex = r"\{(.*?)\}"

for song in song_sections[0]["songs"]:
    with open(song["path"]) as f:
        # print (re.findall(regex,f.read(),re.MULTILINE))
        matches = re.findall(regex,f.read())
        # pp.pprint(matches)
        # important! key is not real key when using capo
        # when capo or custom columns key are searched my break condition will almost always never early quit
        missing_keys = ["title","artist","key","capo","columns_a5"]
        metadata = {}
        for match in matches:
            # print(song["name"], match)
            # remove found key but iterate over copy (by using slicing syntax): 
            for i, key in enumerate(missing_keys[:]):
                if match.startswith(key):
                    metadata[key] = match[len(f"{key}:"):].strip()
                    # print("added: ", song["name"], match)
                    del missing_keys[i]
                    # print(missing_keys)
                    break
            if not missing_keys: break
        song["metadata"] = metadata
           
pp.pprint(song_sections[0]["songs"])

### info/tips

- subprocess tutorial: https://youtu.be/2Fp1N6dof0Y
- using chordpro cli: https://www.chordpro.org/chordpro/using-chordpro/

In [ ]:
# song_sections

In [ ]:
config_path_a5 = os.path.normpath('../settings/chordpro-configs/a5-2column.json')
config_force_one_column_path = os.path.normpath('../settings/chordpro-configs/force_one_column.json')

pdf_setting = {
    "paper_size": "a5"
}

for section in song_sections:
    section_output_dir_path = os.path.join('../data/output/chordpro-pdfs/sections',song_sections[0]['name'])
    if not os.path.exists(section_output_dir_path):
        os.makedirs(section_output_dir_path)
    
    for song in section['songs']:
        chordpro_file_path = song['path']
        pdf_file_path = os.path.join('../data/output/chordpro-pdfs/sections',section['name'],os.path.splitext(song['name'])[0] + '.pdf')
        
        command = ['chordpro']
        
        if pdf_setting["paper_size"] == "a5":
            command.append(f"--config={config_path_a5}")
        if "columns_a5" in song["metadata"] and song["metadata"]["columns_a5"] == '1':
            command.append(f"--config={config_force_one_column_path}")
        
        command.extend([f"--output={pdf_file_path}",chordpro_file_path])
                       
        result = subprocess.run(command,capture_output=True)
        song['chordpro_output'] = {'returncode': result.returncode, 'stdout': result.stdout.decode(),'stderr': result.stderr.decode()}
        song['pdf_file_path'] = pdf_file_path
        
    

In [ ]:
# chordpro_output = song_sections[0]['songs'][0]['chordpro_output']

# chordpro_output

# print(chordpro_output["stderr"])

# pp.pprint(song_sections)

In [ ]:
song_sections_info_dict_pickle_path = os.path.join('../data/output/chordpro-pdfs/sections','song_sections_data.pickle')

with open(song_sections_info_dict_pickle_path, 'wb') as handle:
    pickle.dump(song_sections, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(song_sections_info_dict_pickle_path, 'rb') as handle:
    loaded_song_sections =  pickle.load(handle)

In [ ]:
print(song_sections == loaded_song_sections)

### backup

In [ ]:
result

In [ ]:
result.returncode
result.stdout
result.stderr